In [1]:
import os
import sys
import ipynbname
import numpy as np
sys.path.append('./src')


In [2]:
current_file_path = '/local/zzx/code/BioSeq2Seq'
sample_path = os.path.abspath(os.path.join(current_file_path, 'test_samples/'))
genome_region_path = os.path.abspath(os.path.join(current_file_path, 'genome_regions/'))

chr_list = ['chr22']
window_size = 128

# 1. evaluation of Histone Modification results

In [3]:
histone_pred = os.path.join(sample_path, 'histone/H3k4me1.pred.chr22.bw')
histone_exper = os.path.join(sample_path, 'histone/H3k4me1.exper.chr22.bigWig')
peak_exper = os.path.join(sample_path, 'histone/H3k4me1.bed')

(1) Whole Genome correlation

In [4]:
from HistoneModification.evaluation.correlation.Whole_Genome.corr_genome_wide import correlation_genome_wide

In [6]:
pearson, spearman = correlation_genome_wide(
                                        histone_pred,
                                        histone_exper,
                                        chr_list,
                                        window_size
                                        )
print('Pearson Correlation is %.4f, Spearman Correlation is %.4f' % (pearson, spearman))

Pearson Correlation is 0.7116, Spearman Correlation is 0.7292


(2) Correlation near Functional Elements

In [7]:
from HistoneModification.evaluation.correlation.base_FE.corr_base_fe import correlation_base_functional_elements

In [8]:
fe_file = os.path.join(genome_region_path, 'FE_file/GM12878/promoter.bed')
pearson, spearman = correlation_base_functional_elements(
                                                    histone_pred,
                                                    histone_exper,
                                                    fe_file,
                                                    length=1280,
                                                    include_chr=chr_list,
                                                    window_size=window_size,
                                                    )

print('Pearson Correlation is %.4f, Spearman Correlation is %.4f' % (pearson, spearman))

Pearson Correlation is 0.7477, Spearman Correlation is 0.7886


(3) Correlation base tss

In [28]:
from HistoneModification.evaluation.correlation.base_TSS.corr_base_tss import call_correlation_base_tss

In [31]:
peak_dir = os.path.join(genome_region_path, 'histone_peak_based_TSS/GM12878/near')
pearson_1k, spearman_1k, pearson_10k, spearman_10k, pearson_30k, spearman_30k, pearson_30k_, spearman_30k_ = call_correlation_base_tss(
                                                                                                                peak_dir,
                                                                                                                histone_pred,
                                                                                                                histone_exper,
                                                                                                                chr_list,
                                                                                                                length=1280,
                                                                                                            )



H3k4me1
--------------------------------------------------
0-1k pearson: 0.6329
0-1k spearman 0.6855
1-10k pearson 0.5543
1-10k spearman 0.557
10-30k pearson 0.5249
10-30k spearman 0.4995
30k++ pearson 0.4415
30k++ spearman 0.378


(4) MSE

In [34]:
from HistoneModification.evaluation.MSE.mse_genome_wide import get_genome_wide_mse

In [35]:
fe_dir = os.path.join(genome_region_path, 'FE_file/GM12878')
mseGlobal, mseGene, mseProm, mseEnh, mseObs, mseImp = get_genome_wide_mse(
                                                                        histone_pred,
                                                                        histone_exper,
                                                                        fe_dir=fe_dir,
                                                                        resolution=1280,
                                                                        include_chr=chr_list
                                                                        )

mseGlobal: 11.10468
mseGene: 29.94
mseProm: 38.83
mseEnh: 113.18
mseObs: 272.6
mseImp: 125.58


(5) AUC, AUPR

In [8]:
from HistoneModification.evaluation.roc.roc import draw_roc_prc

In [9]:
outdir = os.path.join(sample_path, 'histone')

roc_auc, prc = draw_roc_prc(
                        histone_pred,
                        peak_exper,
                        outdir,
                        window_size=128,
                        include_chr=chr_list
                        )

print('AUC = %.4f, AUPR =  %.4f' % (roc_auc, prc))

AUC = 0.9044, AUPR =  0.6297


# 2. evaluation of Functional Elements results

In [9]:
promoter_pred = os.path.join(sample_path, 'fe/promoter.chr22.bw')
insulator_pred = os.path.join(sample_path, 'fe/insulator.chr22.bw')
polya_pred = os.path.join(sample_path, 'fe/polya.chr22.bw')
genebody_pred = os.path.join(sample_path, 'fe/genebody.chr22.bw')
promoter_label = os.path.join(genome_region_path, 'FE_file/K562/all_promoter.bed')
insulator_label = os.path.join(genome_region_path, 'FE_file/K562/insulator.bed')
polya_label = os.path.join(genome_region_path, 'FE_file/K562/polya.bed')
genebody_label = os.path.join(genome_region_path, 'FE_file/K562/genebody.bed')
raw_genebody_file = os.path.join(genome_region_path, 'FE_file/genebody_7_lines_raw.bed')
outdir = os.path.join(sample_path, 'fe/out')

(1) Classification Performance

In [4]:
from FunctionalElements.evaluation.classification_performance.classification_eva import fe_classfication_evaluation

In [5]:
fe_classfication_evaluation(
                        pred_promoter=promoter_pred,
                        pred_polya=polya_pred,
                        pred_insulator=insulator_pred,
                        pred_genebody=genebody_pred,
                        label_promoter=promoter_label,
                        label_polya=polya_label,
                        label_insulator=insulator_label,
                        label_genebody=genebody_label,
                        raw_genebody_file=raw_genebody_file,
                        outdir=outdir,
                        include_chr=['chr22']
    )

promoter
TP: 809
FP: 112
TN: 856
FN: 66
intersect_label: 1210
non_intersect_label: 81
Acc: 0.903418339663592
Pre: 0.8783930510314875
Recall: 0.9245714285714286
F-Score: 0.9008908685968819
label recall: 0.9372579395817195
polya
TP: 167
FP: 166
TN: 217
FN: 117
intersect_label: 170
non_intersect_label: 157
Acc: 0.5757121439280359
Pre: 0.5015015015015015
Recall: 0.5880281690140845
F-Score: 0.5413290113452188
label recall: 0.5198776758409785
insulator
TP: 394
FP: 631
TN: 812
FN: 214
intersect_label: 395
non_intersect_label: 324
Acc: 0.5880058508044856
Pre: 0.38439024390243903
Recall: 0.6480263157894737
F-Score: 0.4825474586650336
label recall: 0.5493741307371349
genebody
TP: 422
FP: 51
TN: 462
FN: 12
intersect_label: 434
non_intersect_label: 13
Acc: 0.9334741288278775
Pre: 0.8921775898520085
Recall: 0.9723502304147466
F-Score: 0.9305402425578831
label recall: 0.970917225950783
linc rna:  103
rm linc label recall:  0.9622093023255814


(2) Classification Performance of dREG's label

In [3]:
from FunctionalElements.evaluation.classification_performance.dREG_label_eva import eva_base_dREG_label

In [5]:
outdir = os.path.join(sample_path, 'fe/out/tre_evaluation/dREG')
predicted_path = os.path.join(sample_path, 'fe/out/tre_evaluation/positive_promoter.bed')

positive_label_path = os.path.join(genome_region_path, 'FE_file/dREG_label/K562_dREG_positive.bed')
negative_label_path = os.path.join(genome_region_path, 'FE_file/dREG_label/K562_dREG_negative.bed')
eva_base_dREG_label(predicted_path,
                    positive_label_path,
                    negative_label_path,
                    outpath=outdir,
                    include_chr=['chr22']
                    )

dREG_predicted_path = os.path.join(genome_region_path, 'FE_file/K562/all_promoter.bed')
eva_base_dREG_label(dREG_predicted_path,
                    positive_label_path,
                    negative_label_path,
                    outpath=outdir,
                    include_chr=['chr22']
                    )

TP: 524
FP: 341
TN: 4554
FN: 130
intersect_label: 537
non_intersect_label: 155
Acc: 0.9151198414128672
Pre: 0.6057803468208093
Recall: 0.8012232415902141
F-Score: 0.6899275839368005
label recall: 0.7760115606936416
--------------------------------------------------
TP: 531
FP: 424
TN: 4548
FN: 136
intersect_label: 519
non_intersect_label: 173
Acc: 0.9006916119879411
Pre: 0.556020942408377
Recall: 0.7961019490254873
F-Score: 0.654747225647349
label recall: 0.75
--------------------------------------------------


(3) AUC, AUPR

In [3]:
from FunctionalElements.evaluation.roc.fe_roc import draw_roc_prc

In [10]:
outdir = os.path.join(sample_path, 'fe')

roc_auc, prc = draw_roc_prc(
                        promoter_pred,
                        promoter_label,
                        outdir,
                        window_size=128,
                        include_chr=chr_list
                        )

print('AUC = %.4f, AUPR =  %.4f' % (roc_auc, prc))

AUC = 0.9820, AUPR =  0.9114


# 3. evaluation of gene expression results

In [10]:
ge_pred = os.path.join(sample_path, 'rna/GM12878_pred.chr22.bw')
ge_exper = os.path.join(sample_path, 'rna/GM12878.chr22.bw')
ge_exper_peak = os.path.join(genome_region_path, 'FE_file/GM12878/genebody.bed')

(1) correlation

In [7]:
from GeneExpression.evaluation.correlation.correlation import correlation_genome_wide

In [12]:
pearson, spearman = correlation_genome_wide(
                                        ge_pred,
                                        ge_exper,
                                        include_chr=chr_list,
                                        window_size=1280
                                    )
print('Pearson Correlation is %.4f, Spearman Correlation is %.4f' % (pearson, spearman))

Pearson Correlation is 0.6437, Spearman Correlation is 0.8705


(2) AUC, AUPR

In [13]:
from GeneExpression.evaluation.roc.ge_roc import draw_roc_prc

In [29]:
outdir = os.path.join(sample_path, 'rna')

roc_auc, prc = draw_roc_prc(
                        ge_pred,
                        ge_exper_peak,
                        outdir,
                        window_size=128,
                        include_chr=chr_list
                        )

print('AUC = %.4f, AUPR =  %.4f' % (roc_auc, prc))

AUC = 0.8550, AUPR =  0.6728


(3) method comparison can refer to GeneExpression/evaluation/svm/svm_tran_eva.py

# 4. evaluation of TFBS results

In [ ]:
outdir = os.path.join(sample_path, 'TF/out')
tf_pred = os.path.join(sample_path, 'TF/CTCF.chr22.bw')
tf_exper = os.path.join(sample_path, 'TF/CTCF.label.bed')
tf_exper_bw = os.path.join(sample_path, 'TF/CTCF.label.chr22.bw')

(1) AUPR, Precision at 5% Recall

In [4]:
from TFBS.evaluation.AUPR.aupr import call_aupr

In [5]:
call_aupr(tf_pred,
          tf_exper,
          outdir,
          include_chr=chr_list,
          window_size=1000
          )

CTCF
CTCF: AUPR: 0.580265, Precision at 5% Recall: 1.000000


0.5802648407741444

(2) Correlation

In [7]:
from TFBS.evaluation.correlation.correlation import correlation_genome_wide

In [8]:
correlation_genome_wide(
                        tf_pred,
                        tf_exper_bw,
                        include_chr=chr_list,
                        window_size=1280,
                    )

(0.6874, 0.2893)